In [ ]:
from flask import Flask, render_template, request, redirect, url_for
import os
from werkzeug.utils import secure_filename
import json
import torch
from read import run_ocr

app = Flask(__name__)

UPLOAD_FOLDER = '/Users/Smart/UTRNet-High-Resolution-Urdu-Text-Recognition/templates'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

def load_config(config_path='config.json'):
    with open(config_path, 'r') as f:
        return json.load(f)

@app.route('/')
def home():
    arabic_text = "<p style='font-size: 36px; text-align: center;'>بسم الله الرحمن الرحيم</p><p style='font-size: 36px; text-align: center;'>الحمد لله رب العالمين</p><p style='font-size: 36px; text-align: center;'>اللهم صل على سيدنا محمد وعلى آله وسلم</p>"
    return render_template('home.html', text=arabic_text)

@app.route('/upload', methods=['GET', 'POST'])
def upload():
    if request.method == 'POST':
        if 'image' not in request.files:
            return 'No image part'
        file = request.files['image']
        if file.filename == '':
            return 'No selected file'
        if file:
            filename = secure_filename(file.filename)
            image_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
            file.save(image_path)
            
            config = load_config()
            config['image_path'] = image_path
            
            device = torch.device(f"cuda:{config['device_id']}" if torch.cuda.is_available() and config['device_id'] else 'cpu')
            config['device'] = device
            
            class Struct:
                def __init__(self, **entries):
                    self.__dict__.update(entries)
            
            opt = Struct(**config)
            
            # Run OCR using the custom read.py script
            imgt = run_ocr(opt, device)
            return render_template('image_output.html', img_result=imgt)
    return render_template('upload.html')

if __name__ == '__main__':
    app.run(debug=False)
